## Machine Learning - sMRI Kernel

In [ ]:
# Import modules
import pandas as pd
import numpy as np
import pyreadr
import tqdm

import os

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn import metrics

import matplotlib.pyplot as plt

from math import sqrt

In [ ]:
# Data Fetching function
def getData(data):
  return pyreadr.read_r(data)[None]

In [ ]:
# Get data
batch_df = getData("Preprocessed/Batch_sMRI.rds")
sMRI_df = getData("Preprocessed/Kernel_sMRI.rds")
ocd_df = getData("OCD.rds")

In [ ]:
# Preview data
batch_df.head()

In [ ]:
sMRI_df.head()

In [ ]:
ocd_df.head()

In [ ]:
# Run training model

train_coef = []
train_intercept = []

cum_model_coef = []
cum_model_intercept = []
cum_res = []

# Test data Creation
sample_ids_valid = list(batch_df.loc[batch_df['Valid']==1]['SampleID'])
sMRI_np_valid = sMRI_df.loc[sMRI_df['SampleID'].isin(sample_ids_valid)].to_numpy()[:,1:]
ocd_np_valid = np.concatenate(ocd_df.loc[ocd_df['SampleID'].isin(sample_ids_valid)].to_numpy()[:,1:])

for i in range(1,94):
  sample_ids = list(batch_df.loc[batch_df['Train_'+str(i)]==1]['SampleID'])
  sMRI_np = sMRI_df.loc[sMRI_df['SampleID'].isin(sample_ids)].to_numpy()[:,1:]
  ocd_np = np.concatenate(ocd_df.loc[ocd_df['SampleID'].isin(sample_ids)].to_numpy()[:,1:])
  ocd_np[ocd_np==2]=1
  ocd_np = ocd_np.astype('int')
  model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.9, max_iter=5000).fit(sMRI_np,ocd_np)
  train_coef.append(model.coef_)
  train_intercept.append(model.intercept_)

  train_coef_1 = np.concatenate(train_coef)
  train_intercept_1 = np.concatenate(train_intercept)

  #Finding accuracy on each step
  cum_model_coef = np.mean(train_coef_1, axis=0)
  cum_model_intercept = np.mean(train_intercept_1)

  train_y_prob=[]
  for j in range(len(sMRI_np_valid)):
    train_y_prob.append(1/(1+np.exp(-(np.dot(sMRI_np_valid[j],cum_model_coef)+cum_model_intercept))))
  train_y_pred=np.array(train_y_prob)
  train_y_pred[train_y_pred>0.5]=1
  train_y_pred[train_y_pred<=0.5]=0
  train_y_pred = train_y_pred.astype('int64')
  ocd_np_valid = ocd_np_valid.astype('int64')
  acc_score = accuracy_score(ocd_np_valid,train_y_pred)
  tn, fp, fn, tp = confusion_matrix(ocd_np_valid,train_y_pred).ravel()
  cum_res.append([acc_score,tn,fp,fn,tp])
  print("model", i," processed")

#print(train_coef)
#print(train_intercept)
print(cum_res)


In [ ]:
# Plot
plt.figure(figsize=(9,5), tight_layout=True)
plt.ylim(0.4,0.9)

plt.plot([x[0] for x in cum_res])
plt.axhline(0.5, linestyle='--', color='r')

plt.savefig('Preprocessed/KAC_sMRI.svg', bbox_inches='tight')

In [ ]:
# Baseline
baseline = np.array([0 for _ in range(len(train_y_prob))])
# ROC Curve
baseline_fpr,baseline_tpr,_ = metrics.roc_curve(ocd_np_valid,baseline)
predicted_fpr,predicted_tpr,_ = metrics.roc_curve(ocd_np_valid,train_y_prob)

plt.figure(figsize=(9,5), tight_layout=True)

plt.plot(baseline_fpr, baseline_tpr, linestyle='--', label='Baseline')
plt.plot(predicted_fpr, predicted_tpr, marker='.', label='Elastic Net')
plt.text(0.7, 0.2, 'AUC = 0.5904', fontsize=18)

plt.savefig('Preprocessed/ROC_sMRI.svg', bbox_inches='tight')

#AUC Curve
print("AUC: ", metrics.auc(predicted_fpr,predicted_tpr))

In [ ]:
#Precision Recall Curve
predicted_precision, predicted_recall, _ = metrics.precision_recall_curve(ocd_np_valid, train_y_prob)
baseline_pr=len(ocd_np_valid[ocd_np_valid==1])/len(ocd_np_valid)

plt.figure(figsize=(9,5), tight_layout=True)

plt.plot([0, 1], [baseline_pr, baseline_pr], linestyle='--', label='Baseline')
plt.plot(predicted_precision, predicted_recall, marker='.', label='Elastic Net')
plt.text(0.0, 0.55, 'F1 = 0.4051', fontsize=18)

plt.savefig('Preprocessed/PRC_sMRI.svg', bbox_inches='tight')

# F1-Score
print("F1 Score: ", metrics.f1_score(ocd_np_valid, train_y_pred))

In [ ]:
# Confusion matrix calculations
TN, FP, FN, TP = confusion_matrix(ocd_np_valid,train_y_pred).ravel()
print(confusion_matrix(ocd_np_valid,train_y_pred))

P = TP + FN
N = FP + TN
PP = TP + FP
PN = FN + TN

print("Population =", P+N)
print("Prevalence =", P/(P+N))

CK = (2*((TP * TN) - (TN * FP)))/(((TP + FP) * (FP + TN)) + ((TP + FN) * (FN+TN)))
ACC = (TP + TN) / (P + N)
PPV = TP / PP    
FOR = FN / PN
FDR = FP / PP
NPV = TN / PN
TPR = TP / P
FPR = FP / N
FNR = FN / P
TNR = TN / N
LRp = TPR / FPR
LRn = FNR / TNR
MK = PPV + NPV - 1
BM = TPR + TNR - 1
PT = (sqrt(TPR+FPR) - FPR) / (TPR - FPR)
DOR = LRp/LRn
BA = (TPR + TNR) / 2
FS = (2*PPV * TPR) / (PPV + TPR)
FM = sqrt(PPV * TPR)
MCC = sqrt(TPR*TNR*PPV*NPV) - sqrt(FPR*FNR*FDR*FOR)
TS = TP / (TP + FN + FP)
       
print(CK)
print(ACC)
print(PPV)
print(FOR)
print(FDR)
print(NPV)
print(TPR)
print(FPR)
print(TNR)
print(FNR)
print(LRp)
print(LRn)
print(MK)
print(BM)
print(PT)
print(DOR)
print(BA)
print(FS)
print(FM)
print(MCC)
print(TS)

In [ ]:
# Save data
with open('Preprocessed/Coef_sMRI.npy', 'wb') as f:
    np.save(f, train_coef)
with open('Preprocessed/Inter_sMRI.npy', 'wb') as f:
    np.save(f, train_intercept)